# Paleopiezometry based on dynamically recrystallized grain size

The script includes a function for estimating differential stress based on "average" recrystallized grain sizes named ``calc_diffstress()``. This function requires

- defining the mineral phase and the piezometer relation to use,

- entering the (apparent) grain size as the **equivalent circular diameter in microns**,
- measured with a **specific type of "average" with no stereological correction**,
- and set the type of stress, either uniaxial compression/extension or plane stress, for proper stress correction.

For the first requirement, the GrainSizeTools script includes common mineral phases such as quartz, calcite, olivine and albite (more available soon) and a large list of piezometer relations (17 so far). Also, the script facilitates to write ad hoc piezometric relations.

For the second requirement, the function will automatically convert the equivalent circular diameter to linear intercepts where applicable using de Hoff and Rhines (1968) correction. This is, **you don't have to worry about whether the piezometer was originally calibrated using linear intercepts**, always use the equivalent circular diameters in microns.

The third requirement is key for a correct estimation of the differential stress since each paleopiezometer was calibrated for a specific average grain size (e.g. the arithmetic mean, median, or RMS mean) and, hence, **only provides valid results if the same type of average is used**. Also, **you should not use any type of stereological correction for the estimation of the average grain size**, if the author(s) of the piezometer used any type of stereological correction, the average grain size will be automatically corrected by the function. 

The fourth requirement means that the user has to decide whether to correct or not the estimate of the differential stress for plane stress using the correction factor proposed by Paterson and Olgaard (2000). The rationale for this is that the experiments designed to calibrate piezometers are mainly performed in uniaxial compression while natural shear zones approximately behave as plane stress volumes.

Below are examples of how to obtain information about the different piezometers and define these parameters.


## Get information on piezometric relations

You can get information from the console on the different available piezometric relations  just by typing ``piezometers.*()``, where * is the mineral phase, either ``quartz``, ``calcite``, ``olivine``, or ``feldspar``. For example:

In [1]:
# Load the script first (change the path to GrainSizeTools_script.py accordingly!)
%run C:/Users/marco/Documents/GitHub/GrainSizeTools/grain_size_tools/GrainSizeTools_script.py

module plot imported
module averages imported
module stereology imported
module piezometers imported
module template imported

Welcome to GrainSizeTools script
A free open-source cross-platform script to visualize and characterize grain size
population and estimate differential stress via paleopizometers.

Version: v3.0RC0 (2020-04-23)
Documentation: https://marcoalopez.github.io/GrainSizeTools/

Type get.functions_list() to get a list of the main methods



In [2]:
piezometers.quartz()

Available piezometers:
'Cross'
'Cross_hr'
'Holyoke'
'Holyoke_BLG'
'Shimizu'
'Stipp_Tullis'
'Stipp_Tullis_BLG'
'Twiss'


In [3]:
piezometers.quartz('Twiss')

(550,
 0.68,
 'Ensure that you entered the apparent grain size as the arithmeic mean grain size',
 True,
 1.5)

Note that we obtain five different outputs separated by commas which correspond with:
- the constant *B* of the piezometric relation
- the exponent *m* of the piezometric relation
- A warning indicating the average to use with this piezometric relation
- An indication of whether the piezometric relation was established using linear intercepts (if ``False`` the piezometric relation was established using equivalent circular diameters.
- The stereological factor used (if applicable). If ``False``, no stereological correction applies.

## Using the ``calc_diffstress()`` function

Let us first look at the documentation of the:

In [4]:
?calc_diffstress

Signature: calc_diffstress(grain_size, phase, piezometer, correction=False)
Docstring:
Apply different piezometric relations to estimate the differential
stress from average apparent grain sizes. The piezometric relation has
the following general form:

df = B * grain_size**-m

where df is the differential stress in [MPa], B is an experimentally
derived parameter in [MPa micron**m], grain_size is the aparent grain
size in [microns], and m is an experimentally derived exponent.

Parameters
----------
grain_size : positive scalar or array-like
    the apparent grain size in microns

phase : string {'quartz', 'olivine', 'calcite', or 'feldspar'}
    the mineral phase

piezometer : string
    the piezometric relation

correction : bool, default False
    correct the stress values for plane stress (Paterson and Olgaard, 2000)

 References
-----------
Paterson and Olgaard (2000) https://doi.org/10.1016/S0191-8141(00)00042-0
de Hoff and Rhines (1968) Quantitative Microscopy. Mcgraw-Hill. New 

The ``calc_diffstress`` requires three (obligatory) inputs: (1) the average grain size in microns, (2) the mineral phase, and (3) the piezometric relation to use. We provide few examples below:

In [5]:
calc_diffstress(12, phase='quartz', piezometer='Twiss')

differential stress = 83.65 MPa

INFO:
Ensure that you entered the apparent grain size as the arithmeic mean grain size
ECD was converted to linear intercepts using de Hoff and Rhines (1968) correction


The function returns the estimate of the differential stress (in MPa) plus some relevant information about the corrections made and the type of average expected. The differential stress value obtained above...TODO. You can estimate the differential stress or plane strain using the correction proposed by Paterson and Olgaard (2000)

In [6]:
calc_diffstress(12, phase='quartz', piezometer='Twiss', correction=True)

differential stress = 96.59 MPa

INFO:
Ensure that you entered the apparent grain size as the arithmeic mean grain size
ECD was converted to linear intercepts using de Hoff and Rhines (1968) correction


Some paleopiezometers require uncommon averages such as the root mean square or RMS, for example:

In [7]:
piezometers.quartz('Stipp_Tullis')

(669.0,
 0.79,
 'Ensure that you entered the apparent grain size as the root mean square (RMS)',
 False,
 False)

In this case you should estimate the RMS as  
$RMS = \sqrt{\dfrac{1}{n} (x_{1}^2 + x_{2}^2 + ... + x_{n}^2)}$

In [8]:
filepath = 'C:/Users/marco/Documents/GitHub/GrainSizeTools/grain_size_tools/DATA/data_set.txt'
dataset = pd.read_csv(filepath, sep='\t')  # Import the example dataset
dataset['diameters'] = 2 * np.sqrt(dataset['Area'] / np.pi)  # estimate ECD
rms = np.sqrt(np.mean(dataset['diameters']))  # estimate the root mean squared

calc_diffstress(rms, phase='quartz', piezometer='Stipp_Tullis')

differential stress = 164.65 MPa

INFO:
Ensure that you entered the apparent grain size as the root mean square (RMS)


## Estimation of the differential stress using arrays of values

Alternatively, you can use (numpy) arrays as input instead of scalar values to estimate several differential stresses at once. In this case, the ``calc_diffstress()`` function will return a numpy array, so it is generally more useful to store it in a variable as in the example below.

In [9]:
ameans = np.array([12.23, 13.71, 12.76, 11.73, 12.69, 10.67])  # a set of average grain size values
estimates = calc_diffstress(ameans, phase='olivine', piezometer='VanderWal_wet')
estimates

INFO:
Ensure that you entered the apparent grain size as the arithmetic mean in linear scale
ECD was converted to linear intercepts using de Hoff and Rhines (1968) correction
Differential stresses in MPa


array([167.41, 153.66, 162.16, 172.73, 162.83, 185.45])

If the set of estimated values belongs to the same structural element (e.g. different areas of the the same mylonite or different rocks within the same shear zone), you may want to estimate an average differential stress from all the data. The GrainSizeTools script provides a method named ``conf_interval()`` for this.

In [10]:
?conf_interval

Signature: conf_interval(data, confidence=0.95)
Docstring:
Estimate the confidence interval using the t-distribution with n-1
degrees of freedom t(n-1). This is the way to go when sample size is
small (n < 30) and the standard deviation cannot be estimated accurately.
For large datasets, the t-distribution approaches the normal distribution.

Parameters
----------
data : array-like
    the dataset

confidence : float between 0 and 1, optional
    the confidence interval, default = 0.95

Assumptions
-----------
the data follows a normal or symmetric distrubution (when sample size
is large)

call_function(s)
----------------
Scipy's t.interval

Returns
-------
the arithmetic mean, the error, and the limits of the confidence interval
File:      c:\users\marco\documents\github\grainsizetools\grain_size_tools\grainsizetools_script.py
Type:      function


In [11]:
conf_interval(estimates)

 
Mean = 167.37 ± 11.41
Confidence set at 95.0 %
Max / min = 178.79 / 155.96
Coefficient of variation = 6.8 %


(167.37333333333333, 11.412701448126, (155.96063188520733, 178.78603478145934))